In [1]:
import os
import cv2
import face_recognition
from ultralytics import YOLO

# Load known face encodings
known_encodings = []
known_names = []
known_faces_dir = r'C:\Users\alurm\OneDrive\Desktop\jupyter projects\face_recognition_project\known_faces'

for filename in os.listdir(known_faces_dir):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        path = os.path.join(known_faces_dir, filename)
        name = os.path.splitext(filename)[0]

        image = face_recognition.load_image_file(path)
        encodings = face_recognition.face_encodings(image)

        if encodings:
            known_encodings.append(encodings[0])
            known_names.append(name)
        else:
            print(f"⚠️ No face found in {filename}, skipping.")

# Load YOLO model
model = YOLO(r'C:\Users\alurm\OneDrive\Desktop\jupyter projects\headaim\runs\detect\train\weights\best.pt')

# Start webcam
cap = cv2.VideoCapture(0)
print("🎯 Targeting started. Press ESC to exit.")

def face_match_confidence(face_distance):
    """Convert face distance into confidence score (rough approximation)."""
    return round((1.0 - face_distance) * 100, 2)

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Camera error.")
        break

    frame = cv2.flip(frame, 1)
    results = model(frame)[0]  # Detect heads

    aimed = False

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        head_roi = frame[y1:y2, x1:x2]

        if head_roi.size == 0:
            continue

        rgb_face = cv2.cvtColor(head_roi, cv2.COLOR_BGR2RGB)
        face_locs = face_recognition.face_locations(rgb_face)
        face_encs = face_recognition.face_encodings(rgb_face, face_locs)

        if face_encs:
            encoding = face_encs[0]
            distances = face_recognition.face_distance(known_encodings, encoding)

            if len(distances) > 0:
                best_match_index = distances.argmin()
                best_distance = distances[best_match_index]
                name = known_names[best_match_index]
                confidence = face_match_confidence(best_distance)

                # 🎯 Draw laser and name
                cx = x1 + (x2 - x1) // 2
                cy = y1 + int(0.3 * (y2 - y1))

                cv2.circle(frame, (cx, cy), 8, (0, 0, 255), -1)
                h, w = frame.shape[:2]
                cv2.line(frame, (w // 2, h), (cx, cy), (0, 0, 255), 2)

                label = f"{name} ({confidence:.1f}%)"
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

                aimed = True
                break

        # Draw box and label if no match
        if not aimed:
            cv2.rectangle(frame, (x1, y1), (x2, y2), (100, 100, 100), 1)
            cv2.putText(frame, "Unknown", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (150, 150, 150), 1)

    cv2.imshow("🔴 Smart Headshot Aiming", frame)
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()


🎯 Targeting started. Press ESC to exit.

0: 480x640 1 head, 249.0ms
Speed: 9.6ms preprocess, 249.0ms inference, 9.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 head, 589.1ms
Speed: 4.0ms preprocess, 589.1ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 head, 200.7ms
Speed: 4.5ms preprocess, 200.7ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 head, 157.3ms
Speed: 3.4ms preprocess, 157.3ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 head, 101.5ms
Speed: 2.1ms preprocess, 101.5ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 head, 123.2ms
Speed: 56.2ms preprocess, 123.2ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 head, 100.4ms
Speed: 2.3ms preprocess, 100.4ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 head, 93.4ms
Speed: 1.8ms preprocess, 93.4ms inference, 0.8ms p

In [ ]:
import os
import cv2
import face_recognition
from ultralytics import YOLO

# Load known face encodings
known_encodings = []
known_names = []
known_faces_dir = r'C:\Users\alurm\OneDrive\Desktop\jupyter projects\face_recognition_project\known_faces'

for filename in os.listdir(known_faces_dir):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        path = os.path.join(known_faces_dir, filename)
        name = os.path.splitext(filename)[0]

        image = face_recognition.load_image_file(path)
        encodings = face_recognition.face_encodings(image)

        if encodings:
            known_encodings.append(encodings[0])
            known_names.append(name)
        else:
            print(f"⚠️ No face found in {filename}, skipping.")

# Load YOLO model
model = YOLO(r'C:\Users\alurm\OneDrive\Desktop\jupyter projects\headaim\runs\detect\train\weights\best.pt')

# Start webcam
cap = cv2.VideoCapture(0)
print("🎯 Targeting started (Full-frame face detection). Press ESC to exit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Camera error.")
        break

    frame = cv2.flip(frame, 1)

    # Detect heads using YOLO
    yolo_results = model(frame)[0]
    head_boxes = [box.xyxy[0].tolist() for box in yolo_results.boxes]

    # Run face recognition on full frame
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        matches = face_recognition.compare_faces(known_encodings, face_encoding)
        name = "Unknown"

        if True in matches:
            match_index = matches.index(True)
            name = known_names[match_index]

            # Check if face center is inside any YOLO head box
            face_cx = (left + right) // 2
            face_cy = (top + bottom) // 2

            for x1, y1, x2, y2 in head_boxes:
                if x1 <= face_cx <= x2 and y1 <= face_cy <= y2:
                    # 🎯 Draw laser
                    cx = int((x1 + x2) / 2)
                    cy = int(y1 + 0.3 * (y2 - y1))

                    cv2.circle(frame, (cx, cy), 8, (0, 0, 255), -1)
                    h, w = frame.shape[:2]
                    cv2.line(frame, (w // 2, h), (cx, cy), (0, 0, 255), 2)

                    cv2.putText(frame, name, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
                    break  # One headshot per frame

        else:
            # Unknown face box
            cv2.rectangle(frame, (left, top), (right, bottom), (100, 100, 100), 1)
            cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (150, 150, 150), 1)

    cv2.imshow("🔴 Smart Headshot Aiming (Full-frame)", frame)
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()


In [2]:
import os
import cv2
import face_recognition
from ultralytics import YOLO

# Load known face encodings
known_encodings = []
known_names = []
known_faces_dir = r'C:\Users\alurm\OneDrive\Desktop\jupyter projects\face_recognition_project\known_faces'

for filename in os.listdir(known_faces_dir):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        path = os.path.join(known_faces_dir, filename)
        name = os.path.splitext(filename)[0]

        image = face_recognition.load_image_file(path)
        encodings = face_recognition.face_encodings(image)

        if encodings:
            known_encodings.append(encodings[0])
            known_names.append(name)
        else:
            print(f"⚠️ No face found in {filename}, skipping.")

# Load YOLO model
model = YOLO(r'C:\Users\alurm\OneDrive\Desktop\jupyter projects\headaim\runs\detect\train\weights\best.pt')

# Start webcam
cap = cv2.VideoCapture(0)
print("🎯 Targeting started. Press ESC to exit.")

def face_match_confidence(face_distance):
    """Convert face distance into confidence score (rough approximation)."""
    return round((1.0 - face_distance) * 100, 2)

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Camera error.")
        break

    frame = cv2.flip(frame, 1)
    results = model(frame)[0]  # Detect heads

    aimed = False

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        head_roi = frame[y1:y2, x1:x2]

        if head_roi.size == 0:
            continue

        rgb_face = cv2.cvtColor(head_roi, cv2.COLOR_BGR2RGB)
        face_locs = face_recognition.face_locations(rgb_face)
        face_encs = face_recognition.face_encodings(rgb_face, face_locs)

        if face_encs:
            encoding = face_encs[0]
            distances = face_recognition.face_distance(known_encodings, encoding)

            if len(distances) > 0:
                best_match_index = distances.argmin()
                best_distance = distances[best_match_index]
                name = known_names[best_match_index]
                confidence = face_match_confidence(best_distance)

                # 🎯 Draw laser and name
                cx = x1 + (x2 - x1) // 2
                cy = y1 + int(0.3 * (y2 - y1))

              # Draw crosshair
                crosshair_size = 15
                thickness = 2
                color = (0, 0, 255)  # Red
                
                # Horizontal line
                cv2.line(frame, (cx - crosshair_size, cy), (cx + crosshair_size, cy), color, thickness)
                # Vertical line
                cv2.line(frame, (cx, cy - crosshair_size), (cx, cy + crosshair_size), color, thickness)


                label = f"{name} ({confidence:.1f}%)"
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

                aimed = True
                break

        # Draw box and label if no match
        if not aimed:
            cv2.rectangle(frame, (x1, y1), (x2, y2), (100, 100, 100), 1)
            cv2.putText(frame, "Unknown", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (150, 150, 150), 1)

    cv2.imshow("🔴 Smart Headshot Aiming", frame)
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()


🎯 Targeting started. Press ESC to exit.

0: 480x640 1 head, 280.3ms
Speed: 11.5ms preprocess, 280.3ms inference, 9.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 head, 290.0ms
Speed: 5.2ms preprocess, 290.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 head, 128.9ms
Speed: 4.6ms preprocess, 128.9ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 head, 80.7ms
Speed: 1.7ms preprocess, 80.7ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 head, 71.4ms
Speed: 1.2ms preprocess, 71.4ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 head, 73.5ms
Speed: 1.7ms preprocess, 73.5ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 head, 73.6ms
Speed: 2.6ms preprocess, 73.6ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 head, 68.5ms
Speed: 1.9ms preprocess, 68.5ms inference, 0.7ms postproce